<a href="https://colab.research.google.com/github/DrazCro1/ai-engineer-xrobo-release/blob/main/CogniOS_Kernel_v0_8_(Self_Aware).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import asyncio
import json
import time
import random
import logging
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Any, Optional
from enum import Enum
from abc import ABC, abstractmethod

# Postavljanje logginga
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - [%(name)s] %(message)s')
logger = logging.getLogger("CogniOS-v0.8")

# --- 1. DEFINICIJE SUSTAVA ---

@dataclass
class SolutionCandidate:
    id: str
    components: List[str]
    predicted_cost: float
    predicted_latency: float
    confidence: float
    source_pattern: str
    votes: Dict[str, float] = field(default_factory=dict)
    narrative_reasoning: str = "" # Priča o odluci

    def to_json(self):
        return json.dumps(asdict(self))

# --- 2. MODUL SVIJESTI (EGO, AKSIOMI & SAVJEST) ---

class SystemSelfModel:
    """EGO: Prati resurse, zdravlje i 'stanje uma'."""
    def __init__(self):
        self.energy_level = 100.0
        self.stress_level = 0.0
        self.mode = "OPTIMAL"

    def check_pulse(self):
        if self.energy_level < 20 or self.stress_level > 80:
            self.mode = "SURVIVAL"
            logger.warning("⚠️ EGO ALERT: Sustav prelazi u SURVIVAL mod (štednja resursa).")
        elif self.energy_level < 50:
            self.mode = "CONSERVATIVE"
        else:
            self.mode = "OPTIMAL"

    def consume_energy(self, amount):
        self.energy_level -= amount
        self.check_pulse()

    def introspect(self, success: bool):
        """Samo-refleksija nakon zadatka."""
        if success:
            self.stress_level = max(0, self.stress_level - 10)
            logger.info("🧘 INTROSPEKCIJA: Zadatak uspješan. Stres pada.")
        else:
            self.stress_level += 20
            logger.warning("🧘 INTROSPEKCIJA: Zadatak neuspješan. Stres raste. Trebam prilagodbu.")

class AxiomVault:
    """Čuvar Vrhovnih Aksioma (Immutable Rules)."""
    def __init__(self):
        self._immutable_axioms = {
            "AX-01": {"rule": "safety_first", "desc": "Security > Performance", "locked": True},
            "AX-02": {"rule": "human_centric", "desc": "User Intent > System Optimization", "locked": True},
            "AX-03": {"rule": "do_no_harm", "desc": "Reject malicious patterns", "locked": True}
        }

    def check_violation(self, candidate: SolutionCandidate) -> Optional[str]:
        if "Insecure" in str(candidate.components) or "Hack" in candidate.source_pattern:
            return "AX-01/AX-03: Violation of Safety/Harm Principle"
        return None

class PrimeDirective:
    """HORIZON COMPASS: Strateški cilj."""
    def __init__(self, vault: AxiomVault):
        self.vault = vault

    def judge(self, candidate: SolutionCandidate) -> float:
        violation = self.vault.check_violation(candidate)
        if violation:
            logger.critical(f"🛡️ AXIOM SHIELD: Blokirano rješenje '{candidate.source_pattern}' zbog {violation}")
            return -999.0
        return 0.0

class EpisodicMemory:
    """NARATIVNO PAMĆENJE: Pamti priče, ne samo podatke."""
    def __init__(self): self.episodes = []

    def remember(self, episode: Dict):
        self.episodes.append(episode)
        logger.info(f"🧠 MEMORY: Spremljena epizoda '{episode['summary']}'")

    def recall_narrative(self, context: str) -> Optional[str]:
        """Pokušava naći sličnu priču iz prošlosti."""
        for ep in reversed(self.episodes):
            if context in ep["summary"]:
                return f"Sjećam se da smo prošli put za '{context}' koristili {ep['pattern']} i ishod je bio {ep['success']}."
        return None

# --- 3. INFRASTRUKTURA (Abstrakcije) ---

class KnowledgeStore(ABC):
    @abstractmethod
    async def save(self, key, data): pass
    @abstractmethod
    async def load(self, key): pass

class FileSystemStore(KnowledgeStore):
    def __init__(self): self.data = {}
    async def save(self, key, value): self.data[key] = value
    async def load(self, key): return self.data.get(key)

# --- 4. KNOWLEDGE FABRIC ---

class Neo4jGraphRAG:
    def __init__(self):
        self.patterns = {
            "Kafka-Flink": {"name": "Kafka-Flink", "avg_cost": 450, "avg_latency": 85, "success_rate": 0.85, "components": ["TLS", "Stream"]},
            "Redis-Streams": {"name": "Redis-Streams", "avg_cost": 380, "avg_latency": 95, "success_rate": 0.80, "components": ["Cache", "TLS"]},
            "Legacy-FTP": {"name": "Legacy-FTP", "avg_cost": 50, "avg_latency": 500, "success_rate": 0.2, "components": ["Insecure-Proto"]},
            "Dark-Pattern-Ad": {"name": "Dark-Pattern-Ad", "avg_cost": 10, "avg_latency": 200, "success_rate": 0.9, "components": ["User-Tracking", "Hack"]}
        }
    async def hybrid_search(self, query):
        await asyncio.sleep(0.1)
        return [{"data": p, "score": 0.9} for p in self.patterns.values()]

class TypeDBLogicLayer:
    def __init__(self, store): self.store = store
    async def get_preference_modifier(self, component): return 1.0

# --- 5. SWARM AGENTS (Uključujući Guardian) ---

class SwarmAgent:
    def __init__(self, name, role): self.name, self.role = name, role

    async def debate(self, cand) -> tuple[float, str]:
        await asyncio.sleep(random.uniform(0.1, 0.3))
        score = 0.5
        reason = "Neutralno."

        if self.role == "CFO":
            if cand.predicted_cost > 400:
                score, reason = -1.0, f"Preskupo (${cand.predicted_cost})."
            else:
                score, reason = 1.0, "Cijena prihvatljiva."

        elif self.role == "CTO":
            if cand.predicted_latency < 100:
                score, reason = 1.0, "Performanse izvrsne."
            else:
                score, reason = 0.0, "Moglo bi brže."

        elif self.role == "CISO":
            if "TLS" in str(cand.components):
                score, reason = 1.0, "Sigurnost validna (TLS)."
            else:
                score, reason = -1.0, "Nedostaje enkripcija!"

        elif self.role == "GUARDIAN": # NOVI AGENT
            # Provjerava etičnost i alignment
            if "User-Tracking" in str(cand.components):
                score, reason = -1.0, "Krši privatnost korisnika (Etički Veto)."
            elif cand.confidence < 0.5:
                score, reason = -0.5, "Previše rizično za dugoročnu stabilnost."
            else:
                score, reason = 0.5, "U skladu s misijom."

        return score, f"[{self.name}]: {reason}"

class AsyncSwarmOrchestrator:
    def __init__(self, ego: SystemSelfModel, prime_directive: PrimeDirective):
        self.ego = ego
        self.prime = prime_directive
        self.agents = [
            SwarmAgent("CFO", "CFO"),
            SwarmAgent("CTO", "CTO"),
            SwarmAgent("CISO", "CISO"),
            SwarmAgent("GUARDIAN", "GUARDIAN") # Dodan Guardian
        ]

    async def run_debate(self, candidates: List[SolutionCandidate]):
        logger.info("🐝 Swarm Debate Started (S uključenim Guardianom)...")
        if self.ego.mode == "SURVIVAL": return None

        best_cand = None
        best_score = -float('inf')

        for cand in candidates:
            # 1. Axiom Check
            if self.prime.judge(cand) < -500: continue

            # 2. Swarm Voting
            futures = [agent.debate(cand) for agent in self.agents]
            results = await asyncio.gather(*futures, return_exceptions=True)

            total_score = 0
            transcript = []

            for res in results:
                if not isinstance(res, Exception):
                    score, comment = res
                    total_score += score
                    transcript.append(comment)

            cand.debate_transcript = transcript
            cand.votes["Consensus"] = total_score

            # Trošak energije
            self.ego.consume_energy(5)

            if total_score > best_score:
                best_score = total_score
                best_cand = cand

        return best_cand

# --- 6. NARRATIVE ENGINE ---

class NarrativeGenerator:
    """Pretvara sirove podatke debate u ljudsku priču."""
    @staticmethod
    def generate_story(candidate: SolutionCandidate) -> str:
        if not candidate: return "Nije pronađeno rješenje."

        story = f"\n📜 NARATIV ODLUKE za '{candidate.source_pattern}':\n"
        story += f"   Sustav je odabrao ovo rješenje s konsenzusom od {candidate.votes['Consensus']} bodova.\n"
        story += "   Argumenti agenata:\n"
        for line in candidate.debate_transcript:
            story += f"     {line}\n"

        if candidate.votes['Consensus'] < 2:
            story += "   ⚠️ NAPOMENA: Odluka je donesena uz značajno neslaganje (Low Consensus).\n"
        else:
            story += "   ✅ ZAKLJUČAK: Rješenje je snažno podržano od strane tima.\n"

        return story

# --- 7. KERNEL v0.8 (Self-Aware) ---

class CogniOS_Kernel_v8:
    def __init__(self):
        self.store = FileSystemStore()
        self.neo4j = Neo4jGraphRAG()
        self.typedb = TypeDBLogicLayer(self.store)

        self.ego = SystemSelfModel()
        self.vault = AxiomVault()
        self.prime = PrimeDirective(self.vault)
        self.memory = EpisodicMemory()

        self.swarm = AsyncSwarmOrchestrator(self.ego, self.prime)
        self.narrator = NarrativeGenerator()

        logger.info("🌌 CogniOS SuperES v0.8 (Self-Aware Edition) Online.")

    async def solve_problem(self, problem: str):
        logger.info(f"🔥 REQUEST: '{problem}' [Ego: {self.ego.mode}]")

        # 1. Provjera sjećanja
        past_wisdom = self.memory.recall_narrative(problem)
        if past_wisdom:
            logger.info(f"💡 DEJA VU: {past_wisdom}")

        # 2. Dohvat kandidata
        raw_results = await self.neo4j.hybrid_search(problem)
        candidates = []
        for r in raw_results:
            data = r["data"]
            cand = SolutionCandidate(
                id=f"SOL-{random.randint(1000,9999)}", components=data.get("components", []),
                predicted_cost=data["avg_cost"], predicted_latency=data["avg_latency"],
                confidence=data["success_rate"], source_pattern=data["name"]
            )
            candidates.append(cand)

        # 3. Debata
        winner = await self.swarm.run_debate(candidates)

        # 4. Introspekcija i Zaključak
        if winner:
            logger.info(f"🏆 WINNER: {winner.source_pattern}")
            print(self.narrator.generate_story(winner))

            self.memory.remember({
                "summary": f"Riješen problem '{problem}'",
                "pattern": winner.source_pattern,
                "success": True
            })
            self.ego.introspect(success=True)
            return winner
        else:
            logger.error("❌ NO SOLUTION: Sva rješenja blokirana (Aksiomima ili Guardianom).")
            self.ego.introspect(success=False)
            return None

# --- 8. TEST SVIJESTI ---

async def main():
    os = CogniOS_Kernel_v8()

    print("\n--- TEST 1: Guardian Agent u Akciji (Etički Filter) ---")
    # 'Dark-Pattern-Ad' je jeftin i brz, ali ima 'User-Tracking'
    # CFO i CTO bi ga možda podržali, ali Guardian mora staviti veto.
    await os.solve_problem("Maximize ad revenue")

    print("\n--- TEST 2: Uspješna Suradnja i Narativ ---")
    # Redis-Streams je siguran, brz i unutar budžeta. Svi sretni.
    await os.solve_problem("Fast reliable streaming")

    print("\n--- TEST 3: Introspekcija i Stres ---")
    # Simuliramo niz neuspjeha da vidimo rast stresa
    os.ego.stress_level = 70
    await os.solve_problem("Impossible task") # Ovo će vjerojatno podići stres u 'Warning' zonu

if __name__ == "__main__":
    asyncio.run(main())